In [ ]:
from BESTLIB import MatrixLayout

MatrixLayout.map({
    "A": "ScatterPlot",
    "B": "BarPlot",
    "C": "Slider"
})

layout = MatrixLayout("""
AAC
BBC
""")

layout  # Esto se renderiza en HTML + JS


In [ ]:
from BESTLIB import MatrixLayout

# Mapea letras a "widgets"
MatrixLayout.map({
    "A": "<b style='color:blue'>ScatterPlot</b>",
    "B": "<b style='color:red'>BarPlot</b>",
    "C": "<b style='color:green'>Slider</b>"
})

# Construir layout ASCII
layout = MatrixLayout("""
AAC
BBC
""")

layout


: 